In [1]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import pickle

In [2]:
csvs = [x for x in Path('data').iterdir() if 'timeseries' in str(x)]
counts = []
regions = []
results = pd.DataFrame(columns=['new_deaths', 'new_cases', 'new_recover', 'abs_days', 'rel_days'])
for csv in csvs:
    region = str(csv).split('.')[0].split('_')[-1]
    regions.append(region)
    df = pd.read_csv(csv)
    df = df.iloc[:-1]
    df = df[df['cum_cases']>=10]
    df['abs_days'] = df['dates2'].apply(lambda x: (datetime.strptime(x, '%m/%d/%y') - datetime.strptime('01/22/20', '%m/%d/%y')).days)
    df['rel_days'] = df['dates2'].apply(lambda x: (datetime.strptime(x, '%m/%d/%y') - datetime.strptime(df['dates2'].values[0], '%m/%d/%y')).days)
    counts.append(df.shape[0])
    subset = ['new_deaths', 'new_cases', 'new_recover']
    df[subset] = df[subset].clip(lower = 0)
    subset = ['new_deaths', 'new_cases', 'new_recover', 'abs_days', 'rel_days']
    results = results.append(df[subset])

In [3]:
import pickle
with open('data/stacked.pkl', 'wb') as f:
    pickle.dump({'data': results, 'regions': regions, 'counts': counts}, f)